In [ ]:
import os

# Define the language as a single variable
language = 'dan'

# Use the language variable to construct the folder path
folder_path = f'new_translationss/{language}'

# List all files in the directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        print(os.path.join(root, file))


In [ ]:
import os

# Use the language variable to construct the folder path
folder_path = f'new_translationss/{language}'

# Initialize a dictionary to store the line counts for each file
file_line_counts = {}

# Loop through all files in the specified directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # Skip .DS_Store and any other hidden files
        if file == '.DS_Store' or file.startswith('.'):
            continue

        file_path = os.path.join(root, file)
        
        # Count lines in the file, ignoring decode errors
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                line_count = sum(1 for line in f)
            # Store the line count in the dictionary
            file_line_counts[file] = line_count
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

# Check if all files have the same line count
line_counts = list(file_line_counts.values())
all_equal = all(count == line_counts[0] for count in line_counts)

# Print results
if all_equal:
    print("All files have the same number of lines:", line_counts[0])
else:
    print("Mismatch in line counts detected:")
    for file, count in file_line_counts.items():
        print(f"{file}: {count} lines")


In [ ]:
import sacrebleu
import os
import matplotlib.pyplot as plt
plots_folder = os.path.join(folder_path, "plots")
os.makedirs(plots_folder, exist_ok=True)
# Function to compute BLEU score
def compute_bleu(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [ref_file.read().strip().split('\n')]
    
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = hyp_file.read().strip().split('\n')
    
    # Calculate BLEU score
    bleu = sacrebleu.corpus_bleu(hypotheses, references)
    return bleu.score

# Function to compute chrF score
def compute_chrf(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [ref_file.read().strip().split('\n')]
    
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = hyp_file.read().strip().split('\n')
    
    # Calculate chrF score
    chrf = sacrebleu.corpus_chrf(hypotheses, references)
    return chrf.score

# Function to compute and print all metrics and store them for visualization
def test_scores(reference_file, folder_path):
    models = []
    bleu_scores = []
    chrf_scores = []
    
    # Exclude 'eng.txt' and 'tur.txt' from hypothesis files
    excluded_files = {'eng.txt', f'{language}.txt'}
    hypothesis_files = [f for f in os.listdir(folder_path) if f.endswith('.txt') and f not in excluded_files]
    
    # Compute scores for each hypothesis file
    for hypothesis_file in hypothesis_files:
        hyp_path = os.path.join(folder_path, hypothesis_file)
        models.append(hypothesis_file)
        
        bleu_score = compute_bleu(reference_file, hyp_path)
        chrf_score = compute_chrf(reference_file, hyp_path)
        
        bleu_scores.append(bleu_score)
        chrf_scores.append(chrf_score)

        print(f"{hypothesis_file}:")
        print(f"  BLEU score: {bleu_score:.2f}")
        print(f"  chrF score: {chrf_score:.2f}")
    
    # Visualization after score computation
    visualize_scores(models, bleu_scores, chrf_scores)

# Visualization function
def visualize_scores(models, bleu_scores, chrf_scores):
    plt.figure(figsize=(10, 6))

    # Plot BLEU scores
    plt.subplot(2, 1, 1)
    plt.bar(models, bleu_scores, color='skyblue')
    plt.title('BLEU Scores by Model')
    plt.ylabel('BLEU Score')

    # Plot chrF scores
    plt.subplot(2, 1, 2)
    plt.bar(models, chrf_scores, color='lightcoral')
    plt.title('chrF Scores by Model')
    plt.ylabel('chrF Score')

    # Adjust layout and show the plot
    plt.tight_layout()
    plot_path = os.path.join(plots_folder, f"model_scores_{language}.png")
    plt.savefig(plot_path)
    plt.xticks(rotation=45, ha='right')  # Adjust x-axis labels
    plt.show()

# Paths
reference_path = f'new_translationss/{language}/{language}.txt'
folder_path = f'new_translationss/{language}'

# Run the score comparison and visualize results
test_scores(reference_path, folder_path)